<h1 class="title toc-ignore" style="text-align:center;"><strong>
Group assignment: Data analysis and communication
</strong></h1>

<h3 class="subtitle" style="text-align:center;">
    Data Analytics and Visualisation for Businessg<br>
    Imperial College Business School
</h3>

<h4 class="author" style="text-align:center;">
Nicolas Forster (CID _______); Kasra Khani (CID 02370755); Flaviano Moreira (CID 02288876); Leonardo Ramirez-Lopez (CID 02537657); Mark Shahbazi (CID ______)
</h4>

<h4 class="date" style="text-align:center;">September 09, 2024</h4>

# __Load the datasets__

In [143]:
import base64
import requests as rq
import json
from datetime import date
import numpy as np
import pandas as pd
import time
import ast
import math
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
#from quantile_forest import RandomForestQuantileRegressor  # Adjusted import
import pyproj
import math

In [144]:
import pandas as pd

# this is the url where all the files used in this study live
loc = "https://raw.githubusercontent.com/l-ramirez-lopez/data_share/main/"

# the files with the data of the houses on sale
csv_file_sales = "240908_idealista_sales.csv"

# the files with the data of the houses to rent
csv_file_rent = "240908_idealista_rent.csv"

# read the csv files
df_sales_ini = pd.read_csv(loc + csv_file_sales)
df_rent_ini = pd.read_csv(loc + csv_file_rent)

In [ ]:
    
# display the first few rows of each dataframe to confirm successful loading
print("Sales data dimensions")
print(df_sales_ini.shape)
print("\nRent data dimensions")
print(df_rent_ini.shape)


# __Data wrangling__

## Adding the variable `subtype_property`

In [146]:
import ast
import pandas as pd

def extract_subproperty_type(df):
    """
    Extract the subtype of the property type from the 'detailedType' column and store it in a new column named 'subtype_property'.
    This function handles missing or NaN values in the 'detailedType' column.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing a 'detailedType' column, where each entry is a string representation of a dictionary.

    Returns:
    --------
    pandas.DataFrame
        The modified DataFrame with a new column 'subtype_property', containing the extracted property subtype or NaN for missing values.

    Raises:
    -------
    ValueError:
        If the 'detailedType' string cannot be converted to a dictionary.

    KeyError:
        If the 'typology' key is not found in the 'detailedType' dictionary and it is not a NaN value.

    TypeError:
        If the 'detailedType' value is not a dictionary and cannot be processed.
    """
    for index, row in df.iterrows():
        detailed_type = row['detailedType']

        # Check for NaN or missing values and handle them
        if pd.isna(detailed_type):
            df.at[index, 'subtype_property'] = pd.NA
            continue

        # Convert string representation of a dictionary to an actual dictionary
        if isinstance(detailed_type, str):
            try:
                detailed_type = ast.literal_eval(detailed_type)
            except (ValueError, SyntaxError):
                raise ValueError(f"Could not convert 'detailedType' to a dictionary: {detailed_type}")

        # Ensure detailed_type is now a dictionary
        if isinstance(detailed_type, dict):
            typology = detailed_type.get('typology')
            if not typology:
                raise KeyError(f"'typology' not found in detailedType: {detailed_type}")
            df.at[index, 'subtype_property'] = detailed_type.get('subTypology', typology)
        else:
            raise TypeError(f"'detailedType' is not a dictionary and cannot be processed: {detailed_type}")

    return df


In [147]:
# Get the new variable of property subtype and add it to the dataset
#and store it in a new dataframe
df_sales = extract_subproperty_type(df_sales_ini)
df_rent = extract_subproperty_type(df_rent_ini)

## Fxing the price of one of the properties

In [ ]:
# After a manual inspection of houses with pirceByArea above 10000
# we realised tha there was an error on the price of the property
# with code 104562267. We checked at:
# https://www.idealista.com/en/inmueble/104562267/
# and fixed the price information as well as the priceByArea data
# Update the 'price' for the specified propertyCode
df_sales.loc[df_sales['propertyCode'] == 104562267, 'price'] = 380000

# Recalculate 'priceByArea' as price divided by size
df_sales.loc[df_sales['propertyCode'] == 104562267, 'priceByArea'] = df_sales['price'] / df_sales['size']

# Display the updated row to verify
df_sales[df_sales['propertyCode'] == 104562267]

## Exterior variable

In [ ]:
df_sales['propertyType'].unique()

For the `exterior` variable, the missing values for properties of type 'flat', 'duplex', or 'studio' were assumed to be `False` as may property owners may tent to avoid providing details on this if their property do not face the exterior. For properties of type 'penthouse', 'chalet' or 'countryHouse' with missing values in the `exterior` variable, we assigng them a value of `True` as these properties typically face open spaces. 

In [150]:
# For 'flat', 'duplex', or 'studio' types, assume False for missing 'exterior'
df_sales.loc[df_sales['propertyType'].isin(['flat', 'duplex', 'studio']) & df_sales['exterior'].isnull(), 'exterior'] = False
df_rent.loc[df_rent['propertyType'].isin(['flat', 'duplex', 'studio']) & df_rent['exterior'].isnull(), 'exterior'] = False

In [151]:
# For 'penthouse', 'chalet', or 'countryHouse' types, assume True for missing 'exterior'
df_sales.loc[df_sales['propertyType'].isin(['penthouse', 'chalet', 'countryHouse']) & df_sales['exterior'].isnull(), 'exterior'] = True
df_rent.loc[df_rent['propertyType'].isin(['penthouse', 'chalet', 'countryHouse']) & df_rent['exterior'].isnull(), 'exterior'] = True

In [ ]:
df_sales['exterior'].isna().sum()

In [ ]:
df_rent['exterior'].isna().sum()

## Lift variable

In [154]:
# here we directly assume that if "hasLift" is missing is because it is False
# Normally if the lift is not mentioned is because the house does not have one
# as this is a key attribute for sales
df_sales['hasLift'] = df_sales['hasLift'].fillna(False)
df_rent['hasLift'] = df_rent['hasLift'].fillna(False)

## Adding easting and northing

In [155]:
# Define the WGS 84 and UTM Zone 30N coordinate systems
wgs84 = pyproj.CRS("EPSG:4326")
utm30n = pyproj.CRS("EPSG:25830")

# Create a transformer object
transformer = pyproj.Transformer.from_crs(wgs84, utm30n, always_xy=True)

# Function to transform coordinates
def transform_coordinates(row):
    easting, northing = transformer.transform(row['longitude'], row['latitude'])
    return pd.Series({'easting': easting, 'northing': northing})

# Apply the transformation to each row
df_rent[['easting', 'northing']] = df_rent.apply(transform_coordinates, axis=1)
df_sales[['easting', 'northing']] = df_sales.apply(transform_coordinates, axis=1)

## Reviewing for variables with considerable amount of missing values

In [ ]:
# show the variables with that have some NAs for the sales dataset
sales_nas = df_sales.isna().sum()
sales_nas[sales_nas != 0]

In [ ]:
# show the variables that have some NAs for the rent dataset
rent_nas = df_rent.isna().sum()
rent_nas[rent_nas != 0]

## Floor variable


In [158]:
# replacing the bj = bajos with 0 (Ground floor) and en = entresuelo with 10.5 (between ground and first floor)
df_rent['floor'].replace(['en'], 0.5, inplace = True)
df_rent['floor'].replace(['bj'], 0, inplace = True)

In [ ]:
result = df_rent[df_rent['floor'].isna()]['propertyType'].value_counts()
print(result)

In [ ]:
df_rent['floor'].unique()

For imputing missing values in the `floor` variable, we calculated the median floor location for different property types ('penthouse', 'flat', 'studio', 'duplex'). We then used these mean values to impute the missing floor data for properties of the same type. For instance, missing floor values for penthouses were replaced with the median floor of all penthouses. This approach ensures consistency in the data and helps to prevent the loss of information due to missing values.

In [ ]:
df_rent['floor'] = pd.to_numeric(df_rent['floor'], errors='coerce')
median_floor_r_ph = df_rent.loc[df_rent['propertyType'] == 'penthouse', 'floor'].median()
median_floor_r_f = df_rent.loc[df_rent['propertyType'] == 'flat', 'floor'].median()
median_floor_r_st = df_rent.loc[df_rent['propertyType'] == 'studio', 'floor'].median()
median_floor_r_dx = df_rent.loc[df_rent['propertyType'] == 'duplex', 'floor'].median()
print("penthouse =", median_floor_r_ph, "flat =", median_floor_r_f, "studio =", median_floor_r_st, "duplex =", median_floor_r_dx)

In [162]:
# Replace NaN values in df_rent.floor based on df_rent.propertyType
df_rent.loc[(df_rent.floor.isna()) & (df_rent.propertyType == 'penthouse'), 'floor'] = median_floor_r_ph
df_rent.loc[(df_rent.floor.isna()) & (df_rent.propertyType == 'flat'), 'floor'] = median_floor_r_f
df_rent.loc[(df_rent.floor.isna()) & (df_rent.propertyType == 'studio'), 'floor'] = median_floor_r_st
df_rent.loc[(df_rent.floor.isna()) & (df_rent.propertyType == 'duplex'), 'floor'] = median_floor_r_dx
df_rent.loc[(df_rent.floor.isna()) & (df_rent.propertyType == 'chalet'), 'floor'] = 0
df_rent.loc[(df_rent.floor.isna()) & (df_rent.propertyType == 'countryHouse'), 'floor'] = 0

In [ ]:
df_rent['floor'].isna().sum()

In [ ]:
df_rent.floor.value_counts()

In [ ]:
df_sales.floor.unique()

In [ ]:
df_sales.floor.value_counts()

In [167]:
# replacing the bj = bajos with 0 (Ground floor) and en = entresuelo with 0.5 (between ground and first floor), ss and st as -1 for "subsuelo"
df_sales['floor'].replace(['en'], 0.5, inplace=True)
df_sales['floor'].replace(['bj'], 0, inplace=True)
df_sales['floor'].replace(['ss'], -1, inplace=True)
df_sales['floor'].replace(['st'], -1, inplace=True) #we need to check what st

In [ ]:
df_sales.floor.value_counts()

In [ ]:
df_sales['floor'].isna().sum()

In [ ]:
df_sales['floor'] = pd.to_numeric(df_sales['floor'], errors='coerce')
median_floor_s_ph = df_sales.loc[df_sales['propertyType'] == 'penthouse', 'floor'].median()
median_floor_s_f = df_sales.loc[df_sales['propertyType'] == 'flat', 'floor'].median()
median_floor_s_dx = df_sales.loc[df_sales['propertyType'] == 'duplex', 'floor'].median()
median_floor_s_st = df_sales.loc[df_sales['propertyType'] == 'studio', 'floor'].median()

print("flat = ", median_floor_s_f, "penthouse = ", median_floor_s_ph, "duplex = ", median_floor_s_dx, "studio = ", median_floor_s_st)


In [171]:
# Replace NaN values in 'floor' based on 'propertyType'
df_sales.loc[(df_sales.floor.isna()) & (df_sales.propertyType == 'flat'), 'floor'] = median_floor_s_f
df_sales.loc[(df_sales.floor.isna()) & (df_sales.propertyType == 'penthouse'), 'floor'] = median_floor_s_ph
df_sales.loc[(df_sales.floor.isna()) & (df_sales.propertyType == 'duplex'), 'floor'] = median_floor_s_dx
df_sales.loc[(df_sales.floor.isna()) & (df_sales.propertyType == 'studio'), 'floor'] = median_floor_s_st
df_sales.loc[(df_sales.floor.isna()) & (df_sales.propertyType == 'chalet'), 'floor'] = 0
df_sales.loc[(df_sales.floor.isna()) & (df_sales.propertyType == 'countryHouse'), 'floor'] = 0

In [ ]:
df_sales['floor'].isna().sum()


In [ ]:
df_sales.floor.value_counts()

## Variables 'formerPrice', 'priceDropValue', 'priceDropPercentage'

In [174]:
def extract_price_drop_info(row):
  """
  Extracts price drop information from the 'priceInfo' column and creates new columns.

  Args:
    row: A row from the DataFrame.

  Returns:
    A pandas Series containing the extracted price drop information.
  """
  price_info = row['priceInfo']
  if pd.isna(price_info):
    return pd.Series([pd.NA, pd.NA, pd.NA], index=['formerPrice', 'priceDropValue', 'priceDropPercentage'])

  if isinstance(price_info, str):
    try:
      price_info = ast.literal_eval(price_info)
    except (ValueError, SyntaxError):
      return pd.Series([pd.NA, pd.NA, pd.NA], index=['formerPrice', 'priceDropValue', 'priceDropPercentage'])

  if isinstance(price_info, dict):
    price_dict = price_info.get('price')
    if price_dict and isinstance(price_dict, dict):
      price_drop_info = price_dict.get('priceDropInfo')
      if price_drop_info and isinstance(price_drop_info, dict):
        former_price = price_drop_info.get('formerPrice')
        price_drop_value = price_drop_info.get('priceDropValue')
        price_drop_percentage = price_drop_info.get('priceDropPercentage')
        return pd.Series([former_price, price_drop_value, price_drop_percentage], index=['formerPrice', 'priceDropValue', 'priceDropPercentage'])
  return pd.Series([pd.NA, pd.NA, pd.NA], index=['formerPrice', 'priceDropValue', 'priceDropPercentage'])


# Apply the function to create new columns
df_rent[['formerPrice', 'priceDropValue', 'priceDropPercentage']] = df_rent.apply(extract_price_drop_info, axis=1)
df_sales[['formerPrice', 'priceDropValue', 'priceDropPercentage']] = df_sales.apply(extract_price_drop_info, axis=1)

## Updpate properties 101746143 and 32797738

In [175]:
# update status = 2 for the propertyCode = 101746143 in df_sales 101746143 is a new development in Alicante and 32797738 is a property in good conditions
df_sales.loc[df_sales['propertyCode'] == 101746143, 'status'] = 2
df_sales.loc[df_sales['propertyCode'] == 32797738, 'status'] = 1

In [176]:
# update the df_sales['status'] to 0 is 'renew', 1 if 'good' and 2 if 'newdevelopment'
df_sales['status'] = df_sales['status'].replace({'renew': 0, 'good': 1, 'newdevelopment': 2})
df_rent['status'] = df_rent['status'].replace({'renew': 0, 'good': 1, 'newdevelopment': 2})

## Parking variable 

For imputing missing values for the `parking` variable, we assumed that property owners who did not indicate whether the property has parking likely do not have one. Therefore, all missing values were replaced with `False`.


In [177]:
df_sales['parking'] = df_sales['parkingSpace'].apply(lambda x: x.get('hasParkingSpace') if isinstance(x, dict) else None)

In [178]:
import ast

def extract_has_parking_space(parking_string):
    try:
        parking_dict = ast.literal_eval(parking_string)  # Convert the string to a dictionary
        return parking_dict.get('hasParkingSpace')  # Extract the 'hasParkingSpace' value
    except (ValueError, SyntaxError):
        return None  # Return None if conversion or extraction fails

In [179]:
df_sales['parking'] = df_sales['parkingSpace'].apply(extract_has_parking_space)
df_rent['parking'] = df_rent['parkingSpace'].apply(extract_has_parking_space)

In [180]:
df_sales.loc[df_sales['parking'].isna(), 'parking'] = False
df_rent.loc[df_rent['parking'].isna(), 'parking'] = False

## Find duplicated properties

In [ ]:
# find duplicates in df_rent
# (returns only the second and subsequent occurrences of the duplicates)
duplicate_rows_rent = df_rent[df_rent.duplicated()]
print("Duplicate Rows in df_rent:")
print(duplicate_rows_rent)

In [ ]:
# find duplicates in df_sales
duplicate_rows_sales = df_sales[df_sales.duplicated()]
print("Duplicate Rows in df_sales:")
print(duplicate_rows_sales)

In [ ]:
# number of duplicates in df_sales
num_duplicates_sales = df_sales.duplicated().sum()
print(f"There are {num_duplicates_sales} duplicate rows in df_sales.")

In [184]:
# eliminate duplicates in df_rent and df_sales
df_rent = df_rent.drop_duplicates()
df_sales = df_sales.drop_duplicates()

In [ ]:
print(f"New number of rows in df_rent: {len(df_rent)}")
print(f"New number of rows in df_sales: {len(df_sales)}")

In [ ]:
cols_to_check

In [ ]:
# find the rows in df_rent the rows where the data in 55 colums has the same value but not in the columns propertyCode, thumbnail, latitude, longitude, url,description, externalReference

# Select columns to check for duplicates, excluding specified columns
cols_to_check = [col for col in df_rent.columns if col not in ['propertyCode', 'thumbnail', 'latitude', 'longitude', 'url', 'description', 'externalReference']]

# Find rows where the specified columns have the same values
duplicate_rows = df_rent[df_rent.duplicated(subset=cols_to_check, keep=False)]

# Print the duplicate rows
print(duplicate_rows)


In [ ]:
# find the rows in df_rent the rows where the data in 55 colums has the same value but not in the columns propertyCode, thumbnail, latitude, longitude, url,description, externalReference

# Select columns to check for duplicates, excluding specified columns
cols_to_check = [col for col in df_rent.columns if col in ['description', 'bathrooms', 'rooms']]

# Find rows where the specified columns have the same values
duplicate_rows = df_rent[df_rent.duplicated(subset = cols_to_check, keep = False)]

# Print the duplicate rows
duplicate_rows.shape

## Status variable

In [ ]:
# Define a mapping for the replacements
status_mapping = {0: "renew", 1: "good", 2: "newdevelopment"}

# Replace the values in the 'status' column using .loc
df_sales.loc[:, 'status'] = df_sales['status'].replace(status_mapping)

# Display the updated column
print(df_sales['status'])

In [ ]:
df_rent[df_rent['status'] == 0]['propertyCode']

## Duplicated properties

Some properties had identical textual descriptions, with the same number of bathrooms and rooms, although other fields differed. This suggests that some duplicate properties remain in the dataset. However, they were not removed, as it was unclear which of the differing fields contained the most accurate information.

In [ ]:
# Select columns to check for duplicates, excluding specified columns
cols_to_check = [col for col in df_sales.columns if col in ['description', 'bathrooms', 'rooms']]

# Find rows where the specified columns have the same values
duplicate_rows = df_sales[df_sales.duplicated(subset = cols_to_check, keep = False)]
duplicate_rows.shape

# End of data wrangling

At this point we assume the data is cleaned and prepared to conduct modelling analyses. We then proceed to save this data in csv files:
